# Profitable Apps in the iOS App Store and Google Play Store
We are working for a mobile app company that finances its software through in-app-ads. This means our revenue depends on the number of users for each app.

Our goal for this project therefore is to analyze both app stores to understand what type of apps are likely to attract more users.

## Introduction
As of September 2018, there were approximately 2 million iOS apps available on the App Store, and 2.1 million Android apps on Google Play.

Because a data-analysis over four million apps requires significant resources, we'll instead analyze a sample for the fraction of the cost.

- A data set containing data about approximately 10,000 Android apps from Google Play; the data was collected in August 2018. You can download the data set directly from [this link](https://dq-content.s3.amazonaws.com/350/googleplaystore.csv).
- A data set containing data about approximately 7,000 iOS apps from the Apple App Store; the data was collected in July 2017. You can download the data set directly from [this link](https://dq-content.s3.amazonaws.com/350/AppleStore.csv).

We start by reading the files.

In [1]:
from csv import reader

# Apple Store
opened_app_store = open("AppleStore.csv")
read_app_store = reader(opened_app_store)
app_store = list(read_app_store)
app_store_header = app_store[0]
app_store_data = app_store[1:]

# Play Store
opened_play_store = open("googleplaystore.csv")
read_play_store = reader(opened_play_store)
play_store = list(read_play_store)
play_store_header = play_store[0]
play_store_data = play_store[1:]

In order to make things easier, we'll create a function named `explore_data()`. It will print rows repeatedly and in a readable way.

In [2]:
def explore_data(dataset, start, end, rows_and_columns = False):
    data_slice = dataset[start:end]
    for row in data_slice:
        print(row, "\n")
    if rows_and_columns: 
        print("Number of rows: ", len(dataset))
        print("Number of columns: ", len(dataset[0]))

To get an overview, we'll print the first few rows for each dataset as well as the Number of rows and columns. First, let's take a look at the apple app store.

In [3]:
print(app_store_header, "\n")
explore_data(app_store_data, 0, 3, True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic'] 

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1'] 

['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1'] 

['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1'] 

Number of rows:  7197
Number of columns:  16


As we can see, the Apple Store dataset has 7197 rows with 16 columns. The most relevant columns for our task will be `track_name`,  `price`,  `track_name`,  `rating_count_tot`,  `rating_count_ver`,  `user_rating` and `prime_genre`. For a detailed description of what these columns mean, please check the [documentation](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home).

Now let's do the same for the Google Play store dataset.

In [4]:
print(play_store_header, "\n")
explore_data(play_store_data, 0, 3, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'] 

['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up'] 

['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up'] 

Number of rows:  10841
Number of columns:  13


Our play store data houses 10841 rows with 13 columns each. The most relevant columns will be: `App`,  `Category`,  `Rating`,  `Reviews`,  `Prince`,  and `Genres`. For a detailed description of each category, please check out the [documentation](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps) again. 

## Cleaning things up
Before beginning our analysis, we need to make sure our data is valid and workable. Since we offer free apps for an english-speaking audience, we need to remove all non-English and non-free apps from our dataset.

### Deleting wrong data
First of all, the [discussion](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015) on Kaggle shows that certain rows in the android dataset do not contain all the columns (ie they are shorter than others). That would lead to errors. Let's find which rows these are and delete them.

In [5]:
to_delete = False
delete_index = 1000000 # fictional number

for row in play_store_data: 
    if len(row) != len(play_store_header):
        print(row)
        delete_index = play_store_data.index(row)
        print("\nDeleting row", delete_index)
        to_delete = True
        
if to_delete == True:
    del play_store_data[delete_index]
    print("\nDeleted row ", delete_index)

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']

Deleting row 10472

Deleted row  10472


### Removing duplicates
Secondly, we need to remove some duplicate entries in the play store file. For example, there are four instances of `Instagram` in the data.

In [6]:
for app in play_store_data:
    name = app[0]
    if name == "Instagram":
        print(app)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


Now that we see that there are duplicates. How many are there in total?

In [7]:
duplicates_play_store = []
unique_apps_play_store = []

for app in play_store_data:
    name = app[0]
    if name in unique_apps_play_store:
        duplicates_play_store.append(name)
    else:
        unique_apps_play_store.append(name)
        
print("Number of duplicate apps:", len(duplicates_play_store), "\n")
print("Examples of duplicate apps:", unique_apps_play_store[:10])

Number of duplicate apps: 1181 

Examples of duplicate apps: ['Photo Editor & Candy Camera & Grid & ScrapBook', 'Coloring book moana', 'U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'Sketch - Draw & Paint', 'Pixel Draw - Number Art Coloring Book', 'Paper flowers instructions', 'Smoke Effect Photo Maker - Smoke Editor', 'Infinite Painter', 'Garden Coloring Book', 'Kids Paint Free - Drawing Fun']


In order to remove these apps, we need a strategy.

If we look at the Instagram-entries, the main difference are the number of reviews. This suggests that the entries have different timestamps. For the cleanup-process, we will use the entries with the most reviews – which should also be the most recent.

For the removal process we will thus:
- Create a dictionary, where each dictionary key is a unique app name and the corresponding dictionary value is the highest number of reviews of that app.
- Use the information stored in the dictionary and create a new data set, which will have only one entry per app (and for each app, we'll only select the entry with the highest number of reviews).

In [8]:
reviews_max = {}

for row in play_store_data:
    name = row[0]
    n_reviews = float(row[3])
    
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews
        
print("Expected length of the dictionary:", len(play_store_data) - len(duplicates_play_store))
print("Length of the dictionary:", len(reviews_max))

Expected length of the dictionary: 9659
Length of the dictionary: 9659


Now that we have all the maximum numbers of reviews stored in a variable, we can used the `reviews_max` variable to clean up our data.

In [9]:
android_clean = [] # this will be our cleaned dataset
already_added_android = [] # for counting names

for app in play_store_data:
    name = app[0]
    n_reviews = float(app[3])
    
    if n_reviews == reviews_max[name] and name not in already_added_android:
        android_clean.append(app)
        already_added_android.append(name)
        
# now check the dataset
explore_data(android_clean, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'] 

['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up'] 

['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up'] 

Number of rows:  9659
Number of columns:  13


As we can see, the number of rows is valid and we have successfully cleaned our dataset.

### Removing Non-English apps
Another problem we encounter when going through the data is that a vast amount of apps are non-english. As a company solely targeted towards an English speaking audience, we want to remove these apps from our app data since they are part of a completely different market.

We can see those apps in the following rows for example:

In [10]:
print(app_store_data[813][1])
print(android_clean[4412][0])

爱奇艺PPS -《欢乐颂2》电视剧热播
中国語 AQリスニング


Let's clean our datasets from these apps by checking the names for their character code and removing it if it doesn't belong there.

To allow some space for special characters, a string will be considered non-english if more than three characters fall out of order.

In [11]:
def is_english(string):
    non_english_count = 0
    
    for char in string:
        if ord(char) > 127:
            non_english_count += 1
    if non_english_count > 3:
        return False
    else:
        return True

# some tests
print(is_english('Instagram'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

True
False
True
True


Now that we have a working checking-mechanism for the app-language, let's clean both datasets.

In [12]:
ios_en = []
android_en = []

for app in app_store_data:
    name = app[1]  
    if is_english(name):
        ios_en.append(app)

print("ios dataset after cleanup:")
explore_data(ios_en,0,2,True)
        
for app in android_clean:
    name = app[0]  
    if is_english(name):
        android_en.append(app)
        
print("\nandroid dataset after cleanup:")
explore_data(android_en,0,2,True)

ios dataset after cleanup:
['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1'] 

['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1'] 

Number of rows:  6183
Number of columns:  16

android dataset after cleanup:
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'] 

['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up'] 

Number of rows:  9614
Number of columns:  13


## Isolating free apps
As mentioned in the beginning, we only want to analyze free apps for our purpose. Thus, we need to isolate those in our final datalists.

In [13]:
ios_final = []
android_final = []

for app in ios_en:
    price = app[4]
    if price == "0.0":
        ios_final.append(app)
        
for app in android_en:
    price = app[7]
    if price == "0":
        android_final.append(app)
        
print("final length of ios data:", len(ios_final))
print("final length of android data:", len(android_final))

final length of ios data: 3222
final length of android data: 8864


## Analysis
So far, we have cleaned our data enough to gain some meaningful insights from it. We removed inaccurate data, removed duplicate entries, removed non-English apps and isolated free apps.

Again, our aim is to determine the apps that are likely to attract many users since our apps are financed by advertisements. In detail, our validation strategy for app ideas looks like the following steps:
1. Build a minimal Android version of the app, and add it to Google Play.
2. If the app has a good response from users, we develop it further.
3. If the app is profitable after six months, we build an iOS version of the app and add it to the App Store.

Because we want to add those apps to both stores, we need to find app profiles that are successful on both platforms.

To do that, we'll establish a frequency table of the most common genres using col 11 `prime_genre` of the ios dataset and col 9 `Genres` as well as col 1 `Category` of the android dataset.

We'll build two functions we can use to analyze the frequency tables:
- One function to generate frequency tables that show percentages
- Another function we can use to display the percentages in a descending order

In [14]:
def freq_table(dataset, index):
    table = {}
    total = 0
    
    for row in dataset:
        value = row[index]
        total += 1
        if value in table:
            table[value] += 1
        else:
            table[value] = 1
            
    table_percentages = {}
    for key in table:
        table_percentages[key] = (table[key]/total)*100
        
    return table_percentages
    

# helper function
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)
    
    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ":", entry[0])

We'll start by analyzing the `prime_genre` column of the **Apple App Store**.

In [15]:
display_table(ios_final, 11)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


As we can see, Games make up about 58 % of the entire English-language Apple App Store. Entertainment follows closely at 7.8 % and Photo & Video at 4.9 %. Medical apps, catalogs, navigation and book apps, on the other hand, don't even reach half a percentage point.

It appears that the vast majority of apps in the Apple App Store is designed for entertainment purposes with only a tiny fraction being built for serious usage. However, just because there are a lot of apps in that category doesn't mean they are the most popular apps.

Now if we look at our **android data**:

In [16]:
display_table(android_final, 1)

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

The category column of the Google Play store data shows quite a different picture. Games only make up about 9 % of the offer with `Family` at around 18 % being the largest category. Tools (8 %), Business (5 %) and Lifestyle (4 %) apps are also very present in the Google Play store. Beaty, comics as well as Art and Design, in contrast, only make up a tiny fraction accounting for less then a percentage point each.

If we look at the category Family further, we see that most of the apps in this category are games for kids. Thus making games important nonetheless.

In [17]:
display_table(android_final, 9)

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

The Genre column seems to deliver a much more granular view of the table with lots of games being thrown into different categories. While this might be useful if you want to get into the details, it's not as useful for a broad overview. We'll therefore use the `Category` column for further research.

### Most popular apps
Now that we know the distribution of offerings, let's investigate which apps get downloaded most often.

For the Google Play data, we can use the `Installs` column for that. In the Apple App Store data, we'll have to use a workaround by using the `rating_count_tot` column (since there is no installation number available).

#### Most popular apps in the Apple App Store
Let's start with Apple's App Store:

In [18]:
genres_ios = freq_table(ios_final,11)

for genre in genres_ios:
    total = 0 # number of ratings, not the actual ratings
    len_genre = 0 # number of apps in a genre
    
    for app in ios_final:
        genre_app = app[11]
        if genre_app == genre:
            n_ratings = float(app[5])
            total += n_ratings
            len_genre += 1
    
    avg_n_ratings = total / len_genre
    print(genre, ":", avg_n_ratings)

Utilities : 18684.456790123455
Lifestyle : 16485.764705882353
Education : 7003.983050847458
Reference : 74942.11111111111
Entertainment : 14029.830708661417
Shopping : 26919.690476190477
Health & Fitness : 23298.015384615384
Travel : 28243.8
Catalogs : 4004.0
Book : 39758.5
Medical : 612.0
Business : 7491.117647058823
Finance : 31467.944444444445
Music : 57326.530303030304
Social Networking : 71548.34905660378
Food & Drink : 33333.92307692308
Navigation : 86090.33333333333
Games : 22788.6696905016
Weather : 52279.892857142855
News : 21248.023255813954
Sports : 23008.898550724636
Productivity : 21028.410714285714
Photo & Video : 28441.54375


On average, navigation apps and social media have the highest number of reviews, followed by reference, music, weather and food & drink. Entertainment apps are quite small in comparison. However, there might be some outliers distorting the picture.

In [19]:
for app in ios_final:
    if app[11] == "Navigation":
        print(app[1], ":", app[5])

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


In [20]:
for app in ios_final:
    if app[11] == "Social Networking":
        print(app[1], ":", app[5])

Facebook : 2974676
Pinterest : 1061624
Skype for iPhone : 373519
Messenger : 351466
Tumblr : 334293
WhatsApp Messenger : 287589
Kik : 260965
ooVoo – Free Video Call, Text and Voice : 177501
TextNow - Unlimited Text + Calls : 164963
Viber Messenger – Text & Call : 164249
Followers - Social Analytics For Instagram : 112778
MeetMe - Chat and Meet New People : 97072
We Heart It - Fashion, wallpapers, quotes, tattoos : 90414
InsTrack for Instagram - Analytics Plus More : 85535
Tango - Free Video Call, Voice and Chat : 75412
LinkedIn : 71856
Match™ - #1 Dating App. : 60659
Skype for iPad : 60163
POF - Best Dating App for Conversations : 52642
Timehop : 49510
Find My Family, Friends & iPhone - Life360 Locator : 43877
Whisper - Share, Express, Meet : 39819
Hangouts : 36404
LINE PLAY - Your Avatar World : 34677
WeChat : 34584
Badoo - Meet New People, Chat, Socialize. : 34428
Followers + for Instagram - Follower Analytics : 28633
GroupMe : 28260
Marco Polo Video Walkie Talkie : 27662
Miitomo : 2

In [21]:
for app in ios_final:
    if app[11] == "Reference":
        print(app[1], ":", app[5])

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
WWDC : 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
VPN Express : 14
Real Bike Traffic Rider Virtual Reality Glasses : 8
教えて!goo : 0
Jishokun-Japanese English Dictionary & Translator : 0


As the app list of the categories listed above show, there are a few items that vastly alter the big picture - namely Waze and Google Maps, Facebook as well as the Bible App. They being an outlier create the impression that these categories truly are the top. 

Based on the findings, we can indeed say that the market for iOS apps is greatest in the entertainment section. We have to be careful though, to not choose a niche that doesn't fit our business model (e.g. people don't spend as much time in weather apps as they do in games, for example). Now let's turn to the Google Play Store.

#### Most popular apps in the Google Play Store
When it comes to the Google Play store, we can utilize the number of downloads directly. However, the install numbers don't seem precise enough — we can see that most values are open-ended (100+, 1,000+, 5,000+, etc.):

In [22]:
display_table(android_final,5)

1,000,000+ : 15.726534296028879
100,000+ : 11.552346570397113
10,000,000+ : 10.548285198555957
10,000+ : 10.198555956678701
1,000+ : 8.393501805054152
100+ : 6.915613718411552
5,000,000+ : 6.825361010830325
500,000+ : 5.561823104693141
50,000+ : 4.7721119133574
5,000+ : 4.512635379061372
10+ : 3.5424187725631766
500+ : 3.2490974729241873
50,000,000+ : 2.3014440433213
100,000,000+ : 2.1322202166064983
50+ : 1.917870036101083
5+ : 0.78971119133574
1+ : 0.5076714801444043
500,000,000+ : 0.2707581227436823
1,000,000,000+ : 0.22563176895306858
0+ : 0.04512635379061372
0 : 0.01128158844765343


In [26]:
categories_android = freq_table(android_final,1)

for category in categories_android:
    total = 0
    len_category = 0
    
    for app in android_final:
        category_app = app[1]
        if category_app == category:
            n_installs = app[5]
            n_installs = n_installs.replace('+','')
            n_installs = n_installs.replace(',','')
            total += float(n_installs)
            len_category += 1
    avg_n_installs = total/len_category
    print(category, ":", avg_n_installs)

AUTO_AND_VEHICLES : 647317.8170731707
PERSONALIZATION : 5201482.6122448975
WEATHER : 5074486.197183099
MAPS_AND_NAVIGATION : 4056941.7741935486
ART_AND_DESIGN : 1986335.0877192982
SOCIAL : 23253652.127118643
HOUSE_AND_HOME : 1331540.5616438356
FAMILY : 3695641.8198090694
COMICS : 817657.2727272727
MEDICAL : 120550.61980830671
LIBRARIES_AND_DEMO : 638503.734939759
DATING : 854028.8303030303
SHOPPING : 7036877.311557789
BUSINESS : 1712290.1474201474
FOOD_AND_DRINK : 1924897.7363636363
EVENTS : 253542.22222222222
FINANCE : 1387692.475609756
BOOKS_AND_REFERENCE : 8767811.894736841
GAME : 15588015.603248259
COMMUNICATION : 38456119.167247385
PARENTING : 542603.6206896552
PHOTOGRAPHY : 17840110.40229885
SPORTS : 3638640.1428571427
PRODUCTIVITY : 16787331.344927534
BEAUTY : 513151.88679245283
HEALTH_AND_FITNESS : 4188821.9853479853
TOOLS : 10801391.298666667
ENTERTAINMENT : 11640705.88235294
VIDEO_PLAYERS : 24727872.452830188
EDUCATION : 1833495.145631068
TRAVEL_AND_LOCAL : 13984077.710144928

As the data shows, entertainment-focused apps in the categories entertainment, social media or reference and books as well as magazines seem to gain lots of installations. Utility-based apps like navigation also gain lots of downloads.

## Conclusion
In this project, we analyzed data of the Apple App Store and Google Play android store with the goal of recommending an app profile that can be profitable for both markets.

Based on the data collected, entertainment-based apps certainly offer the best basis for our current business model. Concerning both markets and our business model, building simple apps for quick entertainment seem to be the best choice. One could, for example, focus on an app for daily quotes, mini-games or news-/recipe-collections. These kinds apps should also offer enough screen-time to gain revenue from advertisements.